In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load the SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sample sentences for each emotion category
anger_sentences = ["The service was terrible.", "I am disgusted by the food."]
joy_sentences = ["The food was amazing!", "I loved the ambiance."]
surprise_sentences = ["I was shocked at how bad the food was!", "I couldn’t believe how friendly the staff were."]

# Compute embeddings for each category
anger_embeddings = model.encode(anger_sentences, convert_to_tensor=True)
joy_embeddings = model.encode(joy_sentences, convert_to_tensor=True)
surprise_embeddings = model.encode(surprise_sentences, convert_to_tensor=True)

# Calculate pairwise cosine similarities
anger_joy_similarity = util.pytorch_cos_sim(anger_embeddings, joy_embeddings).mean().item()
anger_surprise_similarity = util.pytorch_cos_sim(anger_embeddings, surprise_embeddings).mean().item()
joy_surprise_similarity = util.pytorch_cos_sim(joy_embeddings, surprise_embeddings).mean().item()

print(f"Anger-Joy Similarity: {anger_joy_similarity}")
print(f"Anger-Surprise Similarity: {anger_surprise_similarity}")
print(f"Joy-Surprise Similarity: {joy_surprise_similarity}")

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Collect all embeddings and labels for clustering
all_sentences = anger_sentences + joy_sentences + surprise_sentences
all_embeddings = model.encode(all_sentences)

# Reduce to 2D for visualization using t-SNE
tsne = TSNE(n_components=2, random_state=0)
reduced_embeddings = tsne.fit_transform(all_embeddings)

# Plotting
labels = ['Anger'] * len(anger_sentences) + ['Joy'] * len(joy_sentences) + ['Surprise'] * len(surprise_sentences)
for label in set(labels):
    indices = [i for i, l in enumerate(labels) if l == label]
    plt.scatter(reduced_embeddings[indices, 0], reduced_embeddings[indices, 1], label=label)

plt.legend()
plt.title("t-SNE Visualization of Emotion Embeddings")
plt.show()

In [ ]:
import numpy as np
from transformers import BertTokenizer, BertModel
import torch

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Example sentences for each emotion
anger_sentences = ["The service was terrible and made me angry.", "I felt disgusted by the food quality."]
joy_sentences = ["The food was delightful and made me happy.", "I felt joyful with the excellent service."]
surprise_sentences = ["I was shocked at how bad the food was!", "I was surprised by the quick service."]

# Define a function to compute SRD for a given sentence
def calculate_srd(sentence, target_word):
    # Tokenize the sentence
    inputs = tokenizer(sentence, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    # Get the index of the target word in the tokenized sentence
    target_indices = [i for i, token in enumerate(tokens) if target_word in token]
    
    # Compute SRD by measuring distances between target indices and all other tokens
    srd_scores = []
    for target_index in target_indices:
        distances = [abs(target_index - i) for i in range(len(tokens))]
        srd_scores.append(distances)
    
    # Return average SRD for the target word in this sentence
    avg_srd = np.mean(srd_scores)
    return avg_srd

# Define a function to calculate average SRD across sentences for an emotion category
def average_srd_for_emotion(sentences, target_words):
    srd_values = []
    for sentence in sentences:
        for word in target_words:
            if word in sentence.lower():
                srd = calculate_srd(sentence, word)
                srd_values.append(srd)
    return np.mean(srd_values)

# Target words associated with each emotion
anger_words = ["angry", "disgusted"]
joy_words = ["happy", "joyful"]
surprise_words = ["surprised", "shocked"]

# Calculate SRD for each emotion category
anger_srd = average_srd_for_emotion(anger_sentences, anger_words)
joy_srd = average_srd_for_emotion(joy_sentences, joy_words)
surprise_srd = average_srd_for_emotion(surprise_sentences, surprise_words)

print(f"Average SRD for Anger: {anger_srd}")
print(f"Average SRD for Joy: {joy_srd}")
print(f"Average SRD for Surprise: {surprise_srd}")